<a href="https://colab.research.google.com/github/Dangandy/covid/blob/world-map/eda/covid19_db_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [1]:
pip install flask_sqlalchemy

# Imports

In [0]:
# db model
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

# create
from datetime import datetime, timedelta
import requests

# lstm
import pandas as pd
import numpy as np
import random as rn

# model
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.losses import MeanSquaredLogarithmicError
from sklearn.model_selection import train_test_split

# predict
from keras.models import load_model
import collections

# db model

In [0]:
# load flask
app = Flask(__name__)
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///site.db"
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False

# load db
db = SQLAlchemy(app)

# db models
class Stat(db.Model):
    id = db.Column(db.String(120), primary_key=True)
    country = db.Column(db.String(100), unique=False, nullable=False)
    date = db.Column(db.Date, nullable=False)
    confirmed = db.Column(db.Integer)
    deaths = db.Column(db.Integer)
    recovered = db.Column(db.Integer)
    confirmed_pred = db.Column(db.Integer)
    deaths_pred = db.Column(db.Integer)
    recovered_pred = db.Column(db.Integer)

    def __repr__(self):
        return f"Stat('{self.country}', '{self.date}', '{self.confirmed}', '{self.deaths}', '{self.recovered}',  '{self.confirmed_pred}', '{self.deaths_pred}', '{self.recovered_pred}')"


# create

In [0]:
class Create:
    def __init__(self):
        # variables
        self.url = "https://pomber.github.io/covid19/timeseries.json"

    def extract(self):
        # get json data
        response = requests.get(self.url)
        json = response.json()

        # return
        return json

    def transform(self, json: dict):
        # variables
        countries = json.keys()
        all_data = []

        # loop
        for country in countries:

            # build array of json
            for i, stat in enumerate(json[country]):
                date = datetime.strptime(stat["date"], "%Y-%m-%d").date()
                data = Stat(
                    id=f"{country}{date}",
                    country=country,
                    date= date,
                    confirmed=stat["confirmed"],
                    deaths=stat["deaths"],
                    recovered=stat["recovered"],
                )
                all_data.append(data)

        # return
        return all_data

    def load(self, data):
        # create db
        db.create_all()

        # add to db
        db.session.bulk_save_objects(data)
        db.session.commit()


def main():
    """
    1. Extract JSON from url
    2. Transform JSON into: id, country, confirmed, recovered, deaths
        - id is in the form of {country}{date}
    3. Load object into sqlite database
    """
    create = Create()
    json = create.extract()
    data = create.transform(json)
    create.load(data)


if __name__ == "__main__":
    main()

# Update

In [0]:
class Update(Create):
    def transform(self, json: dict):
        # variables
        countries = json.keys()
        update_data = []

        # we only want to update everything from yesterday onwards..
        update_day = datetime.now().date() + timedelta(days=-1)

        # loop and build array of Stat
        for country in countries:
            for i, stat in enumerate(json[country]):
                date = datetime.strptime(stat["date"], "%Y-%m-%d").date()

                if date >= update_day:
                    update_data.append(
                        Stat(
                            id=f"{country}{date}",
                            country=country,
                            date=date,
                            confirmed=stat["confirmed"],
                            deaths=stat["deaths"],
                            recovered=stat["recovered"],
                        )
                    )

        # return
        return update_data

    def load(self, data):
        """
        insert if doesn't exist, update otherwise
        """
        # loop
        for stat in data:
            query = Stat.query.get({"id": stat.id})
            if query:
                query.confirmed = stat.confirmed
                query.deaths = stat.deaths
                query.recovered = stat.recovered
            else:
                db.session.add(stat)

        # commit
        db.session.commit()


def main():
    """
    1. Extract new data from url
    2. get last 2 days data ( data here is updated hourly )
    3. insert / update into db
    """
    update = Update()
    json = update.extract()
    last_days = update.transform(json)
    update.load(last_days)


if __name__ == "__main__":
    main()

# build lstm model

In [18]:
class Lstm:
    def extract(self) -> pd.DataFrame:
        """
        extract data from database and output into dataframe
        """
        # grab all record from stat table
        df = pd.read_sql_table("stat", "sqlite:///site.db")

        # return
        return df

    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        transforms done to dataframe:
        - calculate the difference of each metric
        - onehotencode countries
        """
        # get diff
        df["confirmed_diff"] = np.where(
            df.country == df.country.shift(), df.confirmed - df.confirmed.shift(), 0
        )
        df["recovered_diff"] = np.where(
            df.country == df.country.shift(), df.recovered - df.recovered.shift(), 0
        )
        df["deaths_diff"] = np.where(
            df.country == df.country.shift(), df.deaths - df.deaths.shift(), 0
        )

        # encode country with pd.dummies
        dummies = pd.get_dummies(df.country)
        dummies["id"] = df.id
        df = pd.merge(df, dummies, on=["id"])

        # return
        return df

    def load(
        self,
        df: pd.DataFrame,
        metric="confirmed",
        win_size=7,
        epochs=5,
        batch_size=32,
        save=False,
    ) -> Sequential:
        """
        load dataframe into sequential
        """
        # variables
        x, y = [], []
        countries = db.session.query(Stat.country).distinct().all()

        # countries come in the form of [('Afghanistan',), ('Albania',), ... ]
        for (country,) in countries:
            country_df = df[df.country == country]
            series = list(country_df[metric])
            for i in range(0, len(series) - win_size):
                end = i + win_size
                series_x, series_y = series[i:end], series[end]
                if series_y:
                    x.append(series_x)
                    y.append(series_y)
        X, y = np.array(x), np.array(y)

        # TTS
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        # preprocess
        X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
        X_val = X_val.reshape(X_val.shape[0], 1, X_val.shape[1])

        # build model
        model = Sequential()
        model.add(
            LSTM(
                100,
                activation="relu",
                input_shape=(1, win_size),
                return_sequences=True,
            )
        )
        model.add(LSTM(150, activation="relu"))
        model.add(Dense(1, activation="relu"))

        # Compile Model
        model.compile(optimizer="adam", loss=MeanSquaredLogarithmicError())

        # Fit Model
        model.fit(
            X_train,
            y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(X_val, y_val),
            verbose=2,
            shuffle=True,
        )

        # Export Model
        if save:
            model.save("lstm_model.h5")


def main():
    """
    run code
    """
    # Set random state for Keras
    np.random.seed(42)
    rn.seed(12345)

    # build model and save it
    model = Lstm()
    df = model.extract()
    df = model.transform(df)
    lstm = model.load(df, save=True)


if __name__ == "__main__":
    main()


Train on 7503 samples, validate on 1876 samples
Epoch 1/5
 - 2s - loss: 1.6263 - val_loss: 0.0317
Epoch 2/5
 - 1s - loss: 0.0385 - val_loss: 0.0302
Epoch 3/5
 - 1s - loss: 0.0372 - val_loss: 0.0304
Epoch 4/5
 - 1s - loss: 0.0365 - val_loss: 0.0289
Epoch 5/5
 - 1s - loss: 0.0361 - val_loss: 0.0297


## predict

In [0]:
class Predict:
    def __init__(self):
        # start date is 7 days ago
        self.today = datetime.now().date()
        self.start_date = self.today + timedelta(days=-7)

    def get_data(self):
        """
        get last "7" days of data
        """
        # get all prediction of country
        result = Stat.query.filter(
            Stat.date >= self.start_date, Stat.date < self.today
        ).all()

        # create a map for each country and their 7 latest record
        memo = collections.defaultdict(list)
        for stat in result:
            memo[stat.country].append(stat.confirmed)

        # return
        countries = memo.keys()
        X = np.array([memo[country] for country in countries])
        X = X.reshape(X.shape[0], 1, X.shape[1])
        return countries, X

    def predict(self, countries, X, model):
        """
        predict, then predict again
        """
        # variables
        data = []

        # predict - we'll be shifting x every iteration because predict output 1 value
        for i in range(7):
            _X = np.array([x[i : 7 + i] for [x] in X])
            _X = _X.reshape(_X.shape[0], 1, _X.shape[1])
            y_pred = model.predict(_X)

            # add new prediction to x
            X = np.array([np.append(x, y_pred[j]) for j, [x] in enumerate(X)])
            X = X.reshape(X.shape[0], 1, X.shape[1])

        # add predictions into database..
        y_pred = [x[7:14] for [x] in X]
        for country, prediction in zip(countries, y_pred):
            for i, pred in enumerate(prediction):
                pred_date = self.today + timedelta(days=i)
                data.append(
                    Stat(
                        id=f"{country}{pred_date}",
                        country=country,
                        date=pred_date,
                        confirmed_pred=int(pred),
                    )
                )

        # return
        return data

    def load(self, data):
        """
        insert if doesn't exist, update otherwise
        """
        # loop
        for stat in data:
            query = Stat.query.get({"id": stat.id})
            if query:
                query.confirmed = stat.confirmed
                query.deaths = stat.deaths
                query.recovered = stat.recovered
                query.confirmed_pred = (
                    stat.confirmed_pred if stat.confirmed_pred else None
                )
                query.deaths_pred = stat.deaths_pred if stat.deaths_pred else None
                query.recovered_pred = (
                    stat.recovered_pred if stat.recovered_pred else None
                )
            else:
                db.session.add(stat)

        # commit
        db.session.commit()


def main():
    """
    run code
    """
    # variables
    predict = Predict()

    # load model
    lstm = load_model("lstm_model.h5")
    countries, X = predict.get_data()
    data = predict.predict(countries, X, lstm)

    # save to db
    predict.load(data)


if __name__ == "__main__":
    main()


### predict china

Here, it should be flattening.. lets see if the algorithm can predict this..

In [21]:
Stat.query.filter_by(country='China').all()

[Stat('China', '2020-01-22', '548', '17', '28',  'None', 'None', 'None'),
 Stat('China', '2020-01-23', '643', '18', '30',  'None', 'None', 'None'),
 Stat('China', '2020-01-24', '920', '26', '36',  'None', 'None', 'None'),
 Stat('China', '2020-01-25', '1406', '42', '39',  'None', 'None', 'None'),
 Stat('China', '2020-01-26', '2075', '56', '49',  'None', 'None', 'None'),
 Stat('China', '2020-01-27', '2877', '82', '58',  'None', 'None', 'None'),
 Stat('China', '2020-01-28', '5509', '131', '101',  'None', 'None', 'None'),
 Stat('China', '2020-01-29', '6087', '133', '120',  'None', 'None', 'None'),
 Stat('China', '2020-01-30', '8141', '171', '135',  'None', 'None', 'None'),
 Stat('China', '2020-01-31', '9802', '213', '214',  'None', 'None', 'None'),
 Stat('China', '2020-02-01', '11891', '259', '275',  'None', 'None', 'None'),
 Stat('China', '2020-02-02', '16630', '361', '463',  'None', 'None', 'None'),
 Stat('China', '2020-02-03', '19716', '425', '614',  'None', 'None', 'None'),
 Stat('Chin

# api methods

Broken up by individual functions because we can't use flask here

Using 'Canada' as default

## Country Stats

In [0]:
# search db
    result = (
        Stat.query.filter(Stat.confirmed != None, Stat.country == 'Canada')
        .order_by(Stat.date.desc())
        .first()
    )

{
"confirmed": result.confirmed,
    "deaths": result.deaths,
    "recovered": result.recovered,
}

## World Stats

In [0]:
# search db
result = (
    db.session.query(
        db.func.max(Stat.confirmed).label("confirmed"),
        db.func.max(Stat.recovered).label("recovered"),
        db.func.max(Stat.deaths).label("deaths"),
    )
    .filter(Stat.confirmed != None)
    .order_by(Stat.date.desc())
    .group_by(Stat.country)
    .all()
)

# get sums because I don't know how to do it in SQLAlchemy..
confirmed = sum(r[0] for r in result)
recovered = sum(r[1] for r in result)
deaths = sum(r[2] for r in result)

# return
{"confirmed": confirmed, "recovered": recovered, "deaths": deaths}

## Country history

In [0]:
# varialbes
array = []

# use filter to get all data
result = Stat.query.filter_by(country='Canada').all()

# build array
for record in result:
    array.append(
        {
            "confirmed": record.confirmed,
            "date": record.date,
            "recovered": record.recovered,
            "deaths": record.deaths,
            "confirmed_pred": record.confirmed_pred,
        }
    )

# return
{"result": array[:3]}

## World History

In [0]:
# variables
array = []

# query
result = (
    db.session.query(
        Stat.country,
        db.func.max(Stat.confirmed).label("confirmed"),
        Stat.recovered,
        Stat.deaths,
        Stat.confirmed_pred,
    )
    .group_by(Stat.country)
    .all()
)

# build array
for record in result:
    array.append(
        {
            "country": record.country,
            "confirmed": record.confirmed,
            "recovered": record.recovered,
            "deaths": record.deaths,
            "confirmed_pred": record.confirmed_pred,
        }
    )

# return
{"result": array[-5:]}

## Get all country names

In [24]:
db.session.query(
    Stat.country.distinct()
).all()

[('Afghanistan'),
 ('Albania'),
 ('Algeria'),
 ('Andorra'),
 ('Angola'),
 ('Antigua and Barbuda'),
 ('Argentina'),
 ('Armenia'),
 ('Australia'),
 ('Austria'),
 ('Azerbaijan'),
 ('Bahamas'),
 ('Bahrain'),
 ('Bangladesh'),
 ('Barbados'),
 ('Belarus'),
 ('Belgium'),
 ('Benin'),
 ('Bhutan'),
 ('Bolivia'),
 ('Bosnia and Herzegovina'),
 ('Brazil'),
 ('Brunei'),
 ('Bulgaria'),
 ('Burkina Faso'),
 ('Cabo Verde'),
 ('Cambodia'),
 ('Cameroon'),
 ('Canada'),
 ('Central African Republic'),
 ('Chad'),
 ('Chile'),
 ('China'),
 ('Colombia'),
 ('Congo (Brazzaville)'),
 ('Congo (Kinshasa)'),
 ('Costa Rica'),
 ("Cote d'Ivoire"),
 ('Croatia'),
 ('Diamond Princess'),
 ('Cuba'),
 ('Cyprus'),
 ('Czechia'),
 ('Denmark'),
 ('Djibouti'),
 ('Dominican Republic'),
 ('Ecuador'),
 ('Egypt'),
 ('El Salvador'),
 ('Equatorial Guinea'),
 ('Eritrea'),
 ('Estonia'),
 ('Eswatini'),
 ('Ethiopia'),
 ('Fiji'),
 ('Finland'),
 ('France'),
 ('Gabon'),
 ('Gambia'),
 ('Georgia'),
 ('Germany'),
 ('Ghana'),
 ('Greece'),
 ('Guatema